In [37]:
from datetime import datetime
import time
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
api_key = 'af9e771c79336538f1f72e176a3991be'

In [64]:
n = []
for i in tqdm(range(200)):
    if (i == 39 or i == 79 or i == 119 or i == 159 or i ==199):
        time.sleep(10)
    movie = requests.get("http://api.themoviedb.org/3/discover/movie?api_key=" + api_key + "&with_genres=35&sort_by=popularity.desc&language=en-US&page=" + str(i+1))
    movie = movie.json()
    for p in movie['results']:
        n.append("https://www.themoviedb.org/movie/" + str(p['id']))

100%|██████████| 200/200 [01:09<00:00,  2.86it/s]


In [74]:
with open('tmbd.txt','w') as f:
    for x in n:
        f.write(x+'\n')

In [75]:
with open('tmbd.txt','r') as f:
    n = f.readlines()

In [70]:
Colnames = ['name', 'year', 'subtext', 'duration', 'genres', 'director', 'stars',]

def obtain_attributes(url):
    source = requests.get(url)
    data = source.text
    soup = BeautifulSoup(data,'html.parser')
    attr ={}
    for p in Colnames:
        attr[p] = ''
    
    title = soup.find("meta",property="og:title")
    attr['name'] = title.get('content')
    
    crew =soup.find_all('li',class_='profile')
    directors = []
    others = []
    for people in crew:
        p = people.find_all('p')
        if len(p) == 2:
            for c in p[1].get_text().split(','):
                if c.strip() == 'Director':
                    directors.append(p[0].get_text())
    attr['director'] = ' & '.join(directors)
    
    subtext = soup.select_one('div.certification span')
    if subtext is not None:
        subtext = subtext.string
    else:
        subtext = 'not available'
    attr['subtext'] = subtext
    
    year = soup.find("div", class_="header_poster_wrapper")
    release = year.find('span',class_='release_date')
    if release is not None:
        attr['year'] = release.get_text().replace('(','').replace(')','')
    else:
        attr['year'] = 'Not Available'
    
    bar = soup.find_all('strong')
    for c in bar:
        if c.text == 'Runtime':
            attr['duration'] = c.next_sibling
    
    genres_all = soup.find('section',class_='genres right_column')
    genres = []
    for c in genres_all.find_all('li'):
        genres.append(c.get_text())
    attr['genres'] = ' & '.join(genres)
    
    casts_source = requests.get(url + '/cast')
    casts = []
    soup_casts = BeautifulSoup(casts_source.text,'html.parser')
    m = soup_casts.find_all('div',class_='info')
    for c in m:
        if c.find('p',class_='character') is not None:
            casts.append(c.find('a',href=True).text)
    attr['stars'] = ' & '.join(casts)
    
    return attr

In [72]:
attributes = []
for i in n:
    attributes.append(obtain_attributes(i))

In [73]:
import csv
with open('tmdb.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=Colnames)
    writer.writeheader()
    for data in attributes:
        writer.writerow(data)